## [ Dataset & DataLoader ]
- 딥러닝에서는 대량의 데이터 학습 시 많은 시간 소요
- 대량 데이터를 일정 크기로 나누어서 학습 진행
- 에포크 : 처음 ~ 끝까지 전체 데이터 학습 횟수 의미
- 배치사이즈 : 실제 학습하는 데이터 양
- Pytorch에서는 DataLoader와 Dataset 클래스
    * 활용해서 학습 데이터 양만큼 로딩 후 학습 진행
- 데이터셋에 맞는 Custom DataSet을 생성해야 함

[1] 모듈로딩 및 데이터 준비

In [1]:
# [1-1] 모듈 로딩
import torch                                            # 텐서 및 일반 수치계산 함수들
import torch.nn as nn                                   # 인공신경망 관련
import torch.nn.functional as F                         # 인공신경망의 함수들 (AF, LF, MF) 관련
import torch.optim as optim                             # 모델 최적화 관련
from torch.utils.data import Dataset, DataLoader      # 학습 데이터 관련

import pandas as pd

In [2]:
#[1-2] 데이터 준비
DATA_FILE=r'C:\Users\KDP-27\Desktop\KDT6\MachineLearning\data\iris.csv'

dataDF=pd.read_csv(DATA_FILE)
dataDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa


[2] 데이터 전용 커스텀 데이터셋 클래스 정의

In [3]:
# 클래스이름 : CustomDataset
# 매개 변수 : feature, target
# 부모클래스 : Dataset
# 오버라이딩 : __init__(), __len__(), __getitem__()
class CustomDataset(Dataset):
    # 초기화 및 인스턴스 변수 설정 메서드
    def __init__(self, feature, target):
        super().__init__()
        self.feature=feature
        self.target=target
        self.nrows = len(feature)

    # 데이터 갯수 반환 메서드
    def __len__(self):
        return self.nrows
    
    # 인덱스에 해당하는 피쳐와 라벨 반환 메서드
    def __getitem__(self, index):
        x = torch.FloatTensor(self.feature.iloc[index].values)
        y = torch.FloatTensor(self.target.iloc[index].values)
        return x, y

In [4]:
# 커스텀데이터셋 인스턴스 생성
featureDF=dataDF.loc[:,:dataDF.columns[-2]]
featureDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


In [5]:
# [방법 1] Series의 accessor 사용 
#dataDF.variety.str.replace() # replace(정규식으로 조건 입력)

# [방법 2] 
dataDF.replace({'Setosa':0,'Versicolor':1,'Virginica':2},inplace=True)

In [6]:
targetDF=dataDF.loc[:,[dataDF.columns[-1]]] # [] 추가해서 DataFrame으로 만들기
targetDF.head(1)

,variety
0,0


In [10]:
# Dataset 인스턴스 생성
irisDS=CustomDataset(featureDF, targetDF)

# Dataloader 인스턴스 생성
irisDL = DataLoader(irisDS, batch_size=30, shuffle=True)

In [11]:
# DataLoader 인스턴스 확인
for x, y in irisDL : 
    print(x,y)
    break

tensor([[6.3000, 3.4000, 5.6000, 2.4000],
        [6.9000, 3.2000, 5.7000, 2.3000],
        [4.9000, 2.4000, 3.3000, 1.0000],
        [5.3000, 3.7000, 1.5000, 0.2000],
        [6.0000, 3.4000, 4.5000, 1.6000],
        [6.8000, 3.0000, 5.5000, 2.1000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [7.6000, 3.0000, 6.6000, 2.1000],
        [5.2000, 4.1000, 1.5000, 0.1000],
        [5.4000, 3.4000, 1.5000, 0.4000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [4.8000, 3.4000, 1.9000, 0.2000],
        [5.6000, 3.0000, 4.1000, 1.3000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [5.5000, 2.5000, 4.0000, 1.3000],
        [6.7000, 3.1000, 5.6000, 2.4000],
        [6.2000, 2.9000, 4.3000, 1.3000],
        [4.4000, 3.0000, 1.3000, 0.2000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [6.6000, 2.9000, 4.6000, 1.3000],
        [4.9000, 3.1000, 1.5000, 0.2000],
        [6.5000, 2.8000, 4.6000, 1.5000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [6.4000, 2.9000, 4.3000, 1

[3] 모델 클래스 정의

In [14]:
# ----------------------------------------------------------------------------------
# 클래스이름 : IrisModel 
# 부모클래스 : Module
# 클래스구조 : ANN
# - 입 력 층 : 입력 4개, 출력 10개 (즉, 뉴런 10개),  AF ReLU
# - 은 닉 층 : 입력 10개, 출력 10개 (즉, 뉴런 30개), AF ReLU
# - 출 력 층 : 입력 30개, 출력 3개 (즉, 뉴런 3개 ==> 다중분류  갯수만큼), AF Softmax
# ----------------------------------------------------------------------------------

class IrisModel(nn.Module):
    # 모델 구조 층 초기화 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hd_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,3)

    # 순전파 기능 메서드
    def forward(self,x):
        # 입력층
        out=self.in_layer(x)
        out=F.relu(out)
        
        # 은닉층
        out=self.hd_layer(out)
        out=F.relu(out)
        
        # 출력층
        out=self.out_layer(out)
        # out=F.log_softmax(out)    # Pytorch에서 다중분류 손실계산 함수에서 내부적 처리 / 2진분류라면 sigmoid함수 사용해야함

        return out




In [15]:
# 모델 인스턴스 생성
model = IrisModel()
model

IrisModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)

In [16]:
# 모델의 shape 체크
model(torch.FloatTensor([[1.,2.,3.,4]]))

tensor([[-0.0301, -0.5898, -0.3968]], grad_fn=<AddmmBackward0>)

In [21]:
test_data=torch.linspace(0,255,28*28).reshape((-1,28)) # 2차원

test_data.shape, test_data.ndim

(torch.Size([28, 28]), 2)

[3] 학습 => 에포크, 배치사이즈, 최적화 인스터스, 손실함수

In [22]:
# 학습 위한 설정 값
EPOCHS = 10
LR=0.001

# 손실함수 인스턴스, 최적화 인스턴스
LOSS_FN = nn.CrossEntropyLoss()
OPTIMIZER=optim.Adam(model.parameters(),lr=LR)

In [28]:
# 학습 진행
for epoch in range(EPOCHS):
    # 데이터 배치 크기만츰 추출
    for x, y in irisDL:
        # 배치 크기만큼 학습
        output = model(x)

        # 손실 계산 => 손실함수마다 자료형과 형태(shape)가 다름 
        #              적합하게 자료형 및 형태 변환 필요!!!
        y=y.long()  # 자료형 변환 (float -> long)
        loss = LOSS_FN(output,y.reshape((-1,))) # 손실함수에 맞게 y의 shape 수정 (2차원 ->> 1차원)

        # 역전파 - 층별 w, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    # EPOCH단위 loss 출력
    print(f'{epoch}/{EPOCHS} loss : {loss}')

0/10 loss : 0.8781315088272095
1/10 loss : 0.8954373002052307
2/10 loss : 0.8693981766700745
3/10 loss : 0.7783583402633667
4/10 loss : 0.7725905179977417
5/10 loss : 0.782888650894165
6/10 loss : 0.7740792036056519
7/10 loss : 0.7692846655845642
8/10 loss : 0.6649473309516907
9/10 loss : 0.6452271342277527
